# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,810.568903,0.483154,1234,811.052057,810.085749,1622.104114,1620.171497
6,606.887073,-0.537867,1234,606.349207,607.424940,1212.698414,1214.849880
7,71.049087,-0.671038,1234,70.378049,71.720126,140.756098,143.440252
8,156.036247,0.098301,1234,156.134548,155.937946,312.269097,311.875893
9,859.730820,0.920999,1234,860.651819,858.809821,1721.303637,1717.619642
10,801.932450,-0.958635,1234,800.973816,802.891085,1601.947632,1605.782170
11,537.181230,-0.763100,1234,536.418131,537.944330,1072.836261,1075.888660
12,587.641777,-0.099132,1234,587.542645,587.740910,1175.085290,1175.481819
13,81.140945,0.219502,1234,81.360446,80.921443,162.720892,161.842886
14,953.058425,0.407471,1234,953.465896,952.650954,1906.931792,1905.301907


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-811.052057,810.085749
6,-606.349207,607.424940
7,-70.378049,71.720126
8,-156.134548,155.937946
9,-860.651819,858.809821
10,-800.973816,802.891085
11,-536.418131,537.944330
12,-587.542645,587.740910
13,-81.360446,80.921443
14,-953.465896,952.650954


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-811.052057,810.085749
6,-606.349207,607.424940
7,-70.378049,71.720126
8,-156.134548,155.937946
9,-860.651819,858.809821
10,-800.973816,802.891085
11,-536.418131,537.944330
12,-587.542645,587.740910
13,-81.360446,80.921443
14,-953.465896,952.650954


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-811.0520568 ,  810.08574857],
       [-606.34920698,  607.42494002],
       [ -70.37804881,   71.72012578],
       [-156.13454834,  155.93794644],
       [-860.65181857,  858.80982123],
       [-800.97381587,  802.89108507],
       [-536.41813051,  537.94433005],
       [-587.54264507,  587.74090958],
       [ -81.36044623,   80.92144296],
       [-953.46589594,  952.65095371]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1620.104114,-1620.171497
6,1210.698414,-1214.849880
7,138.756098,-143.440252
8,310.269097,-311.875893
9,1719.303637,-1717.619642
10,1599.947632,-1605.782170
11,1070.836261,-1075.888660
12,1173.085290,-1175.481819
13,160.720892,-161.842886
14,1904.931792,-1905.301907


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.810569,0.741577
6,0.606887,0.231067
7,0.071049,0.164481
8,0.156036,0.549150
9,0.859731,0.960499
10,0.801932,0.020683
11,0.537181,0.118450
12,0.587642,0.450434
13,0.081141,0.609751
14,0.953058,0.703736


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,810.568903,0.483154
6,606.887073,-0.537867
7,71.049087,-0.671038
8,156.036247,0.098301
9,859.730820,0.920999
10,801.932450,-0.958635
11,537.181230,-0.763100
12,587.641777,-0.099132
13,81.140945,0.219502
14,953.058425,0.407471


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
